<a href="https://colab.research.google.com/github/klane/playground/blob/master/notebooks/pytorch/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,)),  # [0, 1] range => [-1, 1] range
])

mnist_train = torchvision.datasets.MNIST(
    root='./data', download=True, train=True, transform=transform
)

mnist_val = torchvision.datasets.MNIST(
    root='./data', download=True, train=False, transform=transform
)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [3]:
batch_size = 512

train_loader = torch.utils.data.DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    mnist_val, batch_size=batch_size, num_workers=4, pin_memory=True
)

In [4]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.fc1 = nn.Linear(in_features=64 * 5 * 5, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.reshape(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [5]:
def train(net, optim):
    net.train()
    
    for image, label in train_loader:
        # put data onto GPU
        image = image.cuda()
        label = label.cuda()
        
        # clear gradient
        optim.zero_grad()
        
        # forward through the network
        pred = net.forward(image)
        
        # compute loss and gradient
        loss = F.cross_entropy(pred, label)
        loss.backward()
        
        # update parameters
        optim.step()

In [6]:
def evaluate(net):
    total_train = 0
    correct_train = 0
    total_val = 0
    correct_val = 0
    
    net.eval()

    for image, label in train_loader:
        image = image.cuda()
        label = label.cuda()
        
        with torch.no_grad():
            prediction = net.forward(image).argmax(dim=-1)
            total_train += image.size(0)
            correct_train += (prediction == label).sum().item()
    
    for image, label in val_loader:
        # put data onto GPU
        image = image.cuda()
        label = label.cuda()
        
        with torch.no_grad():
            # forward through the network, and get the predicted class
            prediction = net.forward(image).argmax(dim=-1)
            
            total_val += image.size(0)
            correct_val += (prediction == label).sum().item()
            
    return correct_train / total_train, correct_val / total_val

In [7]:
num_epochs = 10
lr = 0.01
net = MyNet().cuda()
optim = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=2, gamma=0.5)

for epoch in range(num_epochs):
    acc_train, acc_val = evaluate(net)
    print('Epoch: {}\tTrain Accuracy: {:.4f}%\tValidation Accuracy: {:.4f}%'.format(epoch, acc_train * 100, acc_val * 100))
    train(net, optim)
    scheduler.step()

acc_train, acc_val = evaluate(net)
print('Done! \tTrain Accuracy: {:.4f}%\tValidation Accuracy: {:.4f}%'.format(acc_train * 100, acc_val * 100))

Epoch: 0	Train Accuracy: 15.1417%	Validation Accuracy: 16.2800%
Epoch: 1	Train Accuracy: 97.5033%	Validation Accuracy: 97.7300%
Epoch: 2	Train Accuracy: 98.0383%	Validation Accuracy: 97.8300%
Epoch: 3	Train Accuracy: 99.0483%	Validation Accuracy: 98.7400%
Epoch: 4	Train Accuracy: 99.1017%	Validation Accuracy: 98.7600%
Epoch: 5	Train Accuracy: 99.3750%	Validation Accuracy: 98.8900%
Epoch: 6	Train Accuracy: 99.4250%	Validation Accuracy: 98.9100%
Epoch: 7	Train Accuracy: 99.5817%	Validation Accuracy: 98.9300%
Epoch: 8	Train Accuracy: 99.5867%	Validation Accuracy: 99.0000%
Epoch: 9	Train Accuracy: 99.6900%	Validation Accuracy: 98.9600%
Done! 	Train Accuracy: 99.7133%	Validation Accuracy: 98.9900%
